In [1]:
import healpy as hp 
import sys
import os
import numpy as np
sys.path.append('../')
import matplotlib.pyplot as plt
import camb
from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2
from taunet.database import *
from taunet.simulation import CMBspectra,CMBmap,FGMap

In [3]:
db = ForegroundDB()

Connecting to the remote database...


In [4]:
db.get_all_freq(['d1','s1'])

['23']

In [5]:
db.check_model_exist(['d1','s1'],23)

True

In [6]:
fg = FGMap()

Connecting to the remote database...


In [7]:
fg.QU(23)

array([[-4.38828368,  2.36953964, -2.39769959, ...,  7.35810329,
        -5.44491922,  5.3434064 ],
       [-5.56488525,  5.57573315, -3.99086102, ...,  1.07468609,
         0.34125323, -3.3377863 ]])